#Cloning the repo & installing requirements

In [ ]:
!git clone https://github.com/ogunlao/saint.git

In [ ]:
import pandas as pd
import numpy as np

In [2]:
!pip3 install -r '/content/saint/requirements.txt' 

     |████████████████████████████████| 10.4MB 5.4MB/s 
     |████████████████████████████████| 808kB 30.3MB/s 
     |████████████████████████████████| 22.3MB 2.3MB/s 
     |████████████████████████████████| 25.9MB 114kB/s 
     |████████████████████████████████| 10.6MB 27.1MB/s 
     |████████████████████████████████| 804.1MB 23kB/s 
     |████████████████████████████████| 276kB 78kB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 10.2MB 12.4MB/s 
     |████████████████████████████████| 829kB 38.3MB/s 
     |████████████████████████████████| 645kB 37.7MB/s 
     |████████████████████████████████| 122kB 38.6MB/s 
     |████████████████████████████████| 1.2MB 31.7MB/s 
     |████████████████████████████████| 1.3MB 23.1MB/s 
     |████████████████████████████████| 143kB 39.6MB/s 
     |████████████████████████████████| 296kB 44.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=a1f6ad83af5fb24d90

#Reading the data

In [40]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amnaahmedelmustafa","key":"080d905c99ffcf8d5814c2008a11f725"}'}

In [41]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle datasets download -d mastmustu/income
! mkdir ~/data
!unzip income.zip -d data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
  Using cached kaggle-1.5.6-py3-none-any.whl
  0% 0.00/590k [00:00<?, ?B/s]
100% 590k/590k [00:00<00:00, 93.9MB/s]
mkdir: cannot create directory ‘/root/data’: File exists
Archive:  income.zip
  inflating: data/test.csv           
replace data/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/train.csv          


In [42]:

train = pd.read_csv('/content/data/train.csv')
##reading test data for predictions
test = pd.read_csv('/content/data/test.csv')

In [14]:
train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income_>50K'],
      dtype='object')

#custom preprocessing

In [43]:
train_y = train[['income_>50K']]
train = train.drop(columns='income_>50K')

In [44]:
#concat train and test to preprocess
df = pd.concat([train, test])


In [21]:
from sklearn import preprocessing
def preprocess(data, cls_token_idx):
    """
    preprocess: function
        this is a function that process the features
        and return the:
        - the processed data in the order of [cls, categorical features, numerical_features]
        - number of numerical features
        - number of categorical features
        - list of cat_len length describes the 
        number of catogeries in each catogrical column 
    """
    # copying the data
    data = data.copy()

    # adding the cls token to beginning of data
    data.insert(loc=cls_token_idx, column='cls', value='cls')

    cat_cols = data.select_dtypes(include=['object', 'category']).columns
    num_cols = [col for col in data.columns if col not in cat_cols]

    # z-transform and add missing value token
    num_data = data[num_cols]
    num_data = (num_data-num_data.mean())/num_data.std()
    # num_data = (num_data-num_data.min())/(num_data.max() - num_data.min()) min-max scaling
    cat_data = data[cat_cols]
    
    # fill missing
    num_data = num_data.fillna(-99999)
    cat_data[pd.isnull(cat_data)]  = 'NaN'

    # label encoding 
    labelencode = preprocessing.LabelEncoder()
    cat_data = cat_data.apply(labelencode.fit_transform)

    # cat columns come first
    new_data = pd.concat([cat_data.astype(np.int32), num_data.astype(np.float32)], axis=1)

    
        
    cats = []

    for cat in cat_data.columns:
        cats.append(len(pd.unique(new_data[cat])))

    return new_data, len(num_data.columns), len(cat_data.columns), cats
    

In [45]:
processed_data,nums,cats,_=preprocess(df,cls_token_idx=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [23]:
#chech catogrical and numerical data
nums,cats,_

(6, 9, [1, 9, 16, 7, 15, 6, 5, 2, 42])

#splitting dataset

In [24]:
def generate_splits(dataset_size,
                      validation_split,
                      random_seed, shuffle_dataset=True,):
    """Generate data samplers for supervised and semi-supervised training """

    # Creating data indices for training and validation splits:
    indices = list(range(dataset_size))

    split_val = int(validation_split * dataset_size)
    
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    val_indices = indices[:split_val] 
    
    train_indices = indices[split_val: ]


    return train_indices, val_indices

In [46]:
train=processed_data.iloc[:len(train)]
test=processed_data.iloc[len(train):]
train_indices,val_indices = generate_splits(len(train),0.25,random_seed=1234)
x_train, y_train = train.iloc[train_indices], train_y.iloc[train_indices]
x_val, y_val = train.iloc[val_indices], train_y.iloc[val_indices]


In [48]:
y_train.values[32967]

array([0])

#Saving csv files

In [47]:
x_train.to_csv('/content/saint/data/train.csv', index=False)
y_train.to_csv('/content/saint/data/train_y.csv', index=False)
x_val.to_csv('/content/saint/data/val.csv', index=False)
y_val.to_csv('/content/saint/data/val_y.csv', index=False)


In [27]:
%cd '/content/saint/'

/content/saint


In [56]:
!python main.py --experiment sup --no_cat 9 --no_num 6 

Global seed set to 1234
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
2021-07-15 22:59:12.315885: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name        | Type              | Params
--------------------------------------------------
0 | transformer | Encoder           | 65.0 K
1 | embedding   | Embedding         | 3.7 K 
2 | fc          | Linear            | 33    
3 | criterion   | BCEWithLogitsLoss | 0     
4 | train_auroc | AUROC             | 0     
5 | train_acc   | Accuracy          | 0     
6 | valid_acc   | Accuracy          | 0     
7 | test_acc    | Accuracy          | 0     
--------------------------------------------------
6

In [ ]:
!ls /content/saint/checkpoints/lightning_logs/version_2/checkpoints/

'epoch=0-step=1030.ckpt'
